In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import pyplot
import math
import random as ran
from scipy.stats import truncnorm
import pandas as pd

/Users/yerin/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
#timeseries data
all_data=pd.read_csv('../data/Timeseries_data_SP500.csv')
date_list=list(set(list(all_data['DATE'])))
for i in range(len(date_list)):
    ak=date_list[i]
    date_list[i]=ak[6:]+str('-')+ak[:2]+'-'+ak[3:5]
date_list.sort()
all_data['DATE']=pd.to_datetime(all_data['DATE'])
all_data=all_data.rename(columns={'NAME':'name','DATE':'date','SEDOL':'sedol','SECTOR':'sector','BETA':'beta','ALPHA_SCORE':'as','BENCH_WEIGHT':'bw',"MCAP_Q":'mq'})
sedol_list=all_data['sedol'].unique().tolist()

dic_data = {k: v for k, v in all_data.groupby('date')}

In [3]:
dic_data[list(dic_data.keys())[0]]['index']=list(dic_data[list(dic_data.keys())[0]].index)

bw=list(dic_data[list(dic_data.keys())[0]]['bw'])
bw=np.array(bw)
beta=list(dic_data[list(dic_data.keys())[0]]['beta'])
beta=np.array(beta)

sector_set = set(list(dic_data[list(dic_data.keys())[0]]['sector']))
sector_list = list(dic_data[list(dic_data.keys())[0]]['sector'])

gbs=dic_data[list(dic_data.keys())[0]][['sector', 'index']].apply(tuple, axis=1)
group_by_sector = [[y[1] for y in gbs if y[0]==x] for x in sector_set]

mq_list=list(dic_data[list(dic_data.keys())[0]]['mq'])
mq_set=set(list(dic_data[list(dic_data.keys())[0]]['mq']))

gbm=dic_data[list(dic_data.keys())[0]][['mq', 'index']].apply(tuple, axis=1)
group_by_mq = [[y[1] for y in gbm if y[0]==x] for x in mq_set]


In [4]:
total_epoch = 200
#batch_size = 5
learning_rate = 0.0002

n_hidden = 256
n_input = len(bw)
n_noise = 1

In [5]:
X = tf.placeholder(tf.float32)
Z = tf.placeholder(tf.float32, [None, n_noise])

G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.5))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.1))
G_b2 = tf.Variable(tf.zeros([n_input]))
c_9=tf.Variable(0.)
c_6=tf.Variable(0.)
c_7=tf.Variable(0.)


In [11]:
def generator(noise_z):
    hidden = tf.nn.relu(
                    tf.matmul(noise_z, G_W1) + G_b1)
    #(3)
    out = tf.nn.relu(
                    tf.matmul(hidden, G_W2) + G_b2)

    #(4) w_i의 총합이 1이 되도록! 
    output=tf.div(
    out, tf.reduce_sum(out)
    )
    
    return output
generator(Z)



<tf.Tensor 'div_2:0' shape=(?, 492) dtype=float32>

In [8]:
def discriminator(inputs):
    
    #(5)
    bww = tf.convert_to_tensor(bw, dtype=tf.float32)
    d = tf.subtract(inputs,bww)
    a_d = tf.abs(d)
    out1 = tf.subtract(tf.constant(0.05,shape=[1,len(bw)]),a_d)
    
    #(8)
    betaa = tf.convert_to_tensor(beta, dtype=tf.float32)
    beta_wi = tf.multiply(inputs,betaa)
    beta_wb = tf.multiply(betaa,bww)
    dbeta = tf.subtract(beta_wi,beta_wb)
    a_dbeta = tf.abs(dbeta)
    out2 = tf.subtract(tf.constant(0.1,shape=[1,len(bw)]),a_dbeta)
    
    #output = tf.add(tf.reduce_sum(out1),tf.reduce_sum(out2))
    output = tf.reduce_sum(out1)
    return output


In [6]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [10]:
G = generator(Z)
loss_G = discriminator(G)

saver = tf.train.Saver()
G_var_list = [G_W1, G_b1, G_W2, G_b2,c_9,c_6,c_7]

train_G = tf.train.AdamOptimizer(learning_rate).minimize(loss_G,
                                                       var_list=G_var_list)


In [49]:
# (9) 자산 갯수 중 60개만 선택해라 !
def con_9(dfd):
    c_9=-60
    for j in range(len(bw)):
        if dfd[0][j]>0:
            c_9=c_9+1
    if c_9<0 : c_9=-c_9
    else: c_9=c_9
        
    return c_9


# (6) sector별 구분
def con_6(dfd):
    c_6=0
    for i in range(len(list(sector_set))):
        aa=-0.1
        for j in group_by_sector[i]:
            aa=aa+abs(dfd[0][j]-bw[j])
        c_6=c_6+aa
    return c_6


# (7) MCAPQ별 구분  
def con_7(dfd):
    c_7=0
    for i in range(len(list(mq_set))):
        aa=-0.1
        for j in group_by_mq[i]:
            aa=aa+abs(dfd[0][j]-bw[j])
        c_7=c_7+aa
    return c_7
    

In [51]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

loss_val_G = 0
best_sol=1000
best_sol_=0
for epoch in range(total_epoch):
    
    noise = get_noise(1, n_noise)
    sol= sess.run(G, feed_dict={Z: noise})
    
#     c_9=con_9(sol)
#     c_6=con_6(sol)
#     c_7=con_7(sol)
    c_9=0
    c_6=0
    c_6=0
        
    loss_G = discriminator(G)+c_9+c_6+c_7 
    _, loss_val_G = sess.run([train_G, loss_G],
                             feed_dict={Z: noise})

#     print(epoch,c_9)
    
    
# Feasibility Check !
    if c_9 <= 10:
        print(epoch,c_9)



# Best Solution ---------------
    if loss_val_G < best_sol :
        best_sol=loss_val_G
        best_sol_=sol[0]
    
    #save_path = saver.save(sess, "./model.ckpt")


    print(loss_val_G)

0 0
24.1413
1 0
24.1257
2 0
24.1602
3 0
24.1533
4 0
24.1444
5 0
24.0912
6 0
24.0844
7 0
24.0724
8 0
24.1065
9 0
24.0625
10 0
24.048
11 0
24.0516
12 0
24.0488
13 0
24.0433
14 0
24.0335
15 0
24.0334
16 0
24.0845
17 0
24.0237
18 0
24.0711
19 0
24.015
20 0
24.0708
21 0
24.0641
22 0
24.0627
23 0
24.0053
24 0
24.0565
25 0
23.9997
26 0
24.0082
27 0
23.9981
28 0
24.045
29 0
24.0419
30 0
23.9962
31 0
23.9952
32 0
24.0336
33 0
24.0245
34 0
24.0247
35 0
24.0268
36 0
23.9867
37 0
23.9887
38 0
23.9829
39 0
23.9799
40 0
23.987
41 0
23.9858
42 0
24.0243
43 0
24.0112
44 0
23.9767
45 0
23.9802
46 0
24.004
47 0
24.0073
48 0
24.0061
49 0
24.0023
50 0
24.0026
51 0
23.9755
52 0
23.9989
53 0
23.9913
54 0
23.9757
55 0
23.9845
56 0
23.9739
57 0
23.9899
58 0
23.9742
59 0
23.9892
60 0
23.9883
61 0
23.9857
62 0
23.9722
63 0
23.9832
64 0
23.9716
65 0
23.9765
66 0
23.9704
67 0
23.9794
68 0
23.9696
69 0
23.975
70 0
23.9783
71 0
23.9684
72 0
23.9677
73 0
23.9663
74 0
23.9773
75 0
23.9739
76 0
23.9765
77 0
23.9646
78

In [ ]:
sol